### Проектирование признаков

* с помощью внешних источников данных;
* из существующего набора данных.

In [294]:
# импортируем необходимые пакеты

import pandas as pd
import seaborn as sns
import statistics
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [295]:
data = pd.read_csv('data/wine_cleared.csv') # чтение данных

In [188]:
display(data.describe())

,Unnamed: 0,points,price
count,129971.000000,129971.000000,129971.000000
mean,64985.000000,88.447138,35.363389
std,37519.540256,3.039730,39.577066
min,0.000000,80.000000,4.000000
25%,32492.500000,86.000000,18.000000
50%,64985.000000,88.000000,28.000000
75%,97477.500000,91.000000,40.000000
max,129970.000000,100.000000,3300.000000


In [189]:
display(data.describe(include=['object']))

,country,description,designation,province,region_1,taster_name,taster_twitter_handle,title,variety,winery
count,129908,129971,129971,129908,129971,129971,129971,129971,129970,129971
unique,43,119955,37980,425,1230,20,16,118840,707,16757
top,US,"Seductively tart in lemon pith, cranberry and ...",unknown,California,unknown,unknown,unknown,Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma...,Pinot Noir,Wines & Winemakers
freq,54504,3,37465,36247,21247,26244,31213,11,13272,222


Новые признаки можно сконструировать из уже существующего набора данных несколькими способами: 

* разбор категорий; 
* разбор даты и времени;
* разбор числовых признаков;
* разбор текста.

**1. Разбор числовых величин**

In [296]:
# округлённая до целого числа цена за бутылку вина
data['price_round'] = data['price'].round().astype(int)

**2. Разбор текста**

In [209]:
# выделим из названия год производства
regex = '\d{4}' # регулярное выражение для нахождения чисел
data['year'] = data['title'].str.findall(regex).str.get(0)
data['year'] = data['year'].apply(lambda x: x if x is np.NaN else int(x))

In [297]:
regex = '\d{4}' # регулярное выражение для нахождения чисел
data['year'] = data['title'].str.findall(regex).str.get(0)
data['year'] = pd.to_datetime(data['year'])
data['years_diff'] = (pd.to_datetime("2022-01-12") - data['year']).dt.days
data['years_diff'].max()

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 1637-01-01 00:00:00 present at position 27

**3. Разбор категорий**

In [210]:
# самая популярная страна-производитель вина — США. 
# Возможно, это не случайность, и факт производства в США влияет на рейтинг вина
data['is_usa'] = data['country'].apply(lambda x: 1 if x == 'US' else 0)
data['is_france'] = data['country'].apply(lambda x: 1 if x == 'France' else 0)
data['is_italy'] = data['country'].apply(lambda x: 1 if x == 'Italy' else 0)

data['old_wine'] = data['year'].apply(lambda x: 1 if x < 2010 else 0)

regex = '\((\w*)\)'
data['locality'] = data['title'].str.findall(regex).str.get(0)

**4. Разбор дат**

In [211]:
def conv(x):
    if np.isnan(x):
        return x
    return pd.Period(year=x, freq="D")

end_date = pd.Period(year=2022, month=1, day=12, freq="D")
start_date = data['year'].apply(conv)
data['years_diff'] = (end_date - start_date).apply(lambda x: 0 if pd.isnull(x) else x.n)

In [212]:
display(data.years_diff.max())

373289

In [204]:
data.year.value_counts().sort_index()

1000.0        2
1070.0        1
1150.0        2
1492.0        6
1503.0        1
          ...  
2015.0    10041
2016.0     3690
2017.0       11
3000.0        1
7200.0       14
Name: year, Length: 91, dtype: int64

In [221]:
data[data.title.find('1000') > -1]

AttributeError: 'Series' object has no attribute 'find'

Данные о звонках колл-центра

* client_id — идентификатор клиента в базе;
* agent_date — время соединения с агентом;
* created_at — время соединения с клиентом (начало разговора);
* end_date — время окончания соединения с клиентом (конец разговора).

In [76]:
# инициализируем информацию о звонках
calls_list = [
    [460, '2013-12-17 04:55:39', '2013-12-17 04:55:44', '2013-12-17 04:55:45'],
    [12, '2013-12-16 20:03:20', '2013-12-16 20:03:22', '2013-12-16 20:07:13'],
    [56, '2013-12-16 20:03:20', '2013-12-16 20:03:20', '2013-12-16 20:05:04'],
    [980, '2013-12-16 20:03:20','2013-12-16 20:03:27', '2013-12-16 20:03:29'],
    [396, '2013-12-16 20:08:27', '2013-12-16 20:08:28','2013-12-16 20:12:03'],
    [449, '2013-12-16 20:03:20', '2013-12-16 20:03:25','2013-12-16 20:05:00'],
    [397, '2013-12-16 20:08:25', '2013-12-16 20:08:27', '2013-12-16 20:09:59'],
    [398, '2013-12-16 20:01:23', '2013-12-16 20:01:23', '2013-12-16 20:04:58'],
    [452, '2013-12-16 20:03:20', '2013-12-16 20:03:21','2013-12-16 20:04:55'],
    [440, '2013-12-16 20:03:20', '2013-12-16 20:04:26', '2013-12-16 20:04:32']
]

calls = pd.DataFrame(calls_list, columns = ['client_id',  'agent_date', 'created_at' ,'end_date'])

# преобразовываем признаки в формат datetime для удобной работы

calls['agent_date'] = pd.to_datetime(calls['agent_date'])
calls['created_at'] = pd.to_datetime(calls['created_at'])
calls['end_date'] = pd.to_datetime(calls['end_date'])

calls.head(3)

,client_id,agent_date,created_at,end_date
0,460,2013-12-17 04:55:39,2013-12-17 04:55:44,2013-12-17 04:55:45
1,12,2013-12-16 20:03:20,2013-12-16 20:03:22,2013-12-16 20:07:13
2,56,2013-12-16 20:03:20,2013-12-16 20:03:20,2013-12-16 20:05:04


In [81]:
calls['duration'] = (calls['end_date'] - calls['created_at']).dt.seconds # длительность разговора
calls['time_connection'] = (calls['created_at'] - calls['agent_date']).dt.seconds # время соединения
calls['is_connection'] = calls['duration'].apply(lambda x: 1 if x > 10 else 0) # факт соединения с клиентом
calls['time_diff'] = (calls['end_date'] - calls['agent_date']).dt.seconds # общее время, потраченное оператором

calls = calls.drop(columns=['agent_date', 'created_at' ,'end_date'], axis=1) # удалим лишние признаки

**Получение данных из внешних источников**

In [65]:
import pandas as pd
country_population = pd.read_csv('data/country_population.csv', sep=';')

country_population.head(3)

,country,population
0,China,"1,411,778,724"
1,India,"1,386,584,581"
2,US,"333,022,386"


In [66]:
data = data.join(country_population.set_index('country'), on='country')

In [67]:
import pandas as pd

country_area = pd.read_csv('data/country_area.csv', sep=';')
country_area.head(3)

,country,area
0,Russia,17075400.0
1,Canada,9984670.0
2,China,9596960.0


In [73]:
data = data.join(country_area.set_index('country'), on='country', rsuffix='_country')

#### Отбор признаков (мультиколлинеарность)

Удаление признаков с очень сильной корреляцией (где коэффициент корреляции +/-0.7 и выше).